In [1]:
from trajectory.heuristic.iql import load_iql
import pandas as pd
import numpy as np
from data_mapping import Data_mapping
import torch

In [2]:
iql_path = '../logs/maze2d-umaze-v1/iql-10e6/final.pt'
iql = load_iql(4, 2, iql_path)

/home/dabai/miniconda3/envs/rl/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df = pd.read_csv('MCTS_Q_map.csv')
df.head()

,x,y,Q
0,5,8,-0.288525
1,4,7,0.272498
2,5,7,0.301793
3,3,7,-0.495502
4,4,8,-0.428744


In [4]:
obs = pd.read_csv('data/obs.csv',usecols=[1,2])
data = obs.values

In [5]:
datam = Data_mapping(10, data)

In [6]:
pos_tok = df.iloc[:,:2].values

In [7]:
pos = datam.reconstruct(pos_tok)

In [39]:
vx = np.arange(-3, 3, 0.5)
vy = np.arange(-3, 3, 0.5)
v = []
for i in vx:
    for j in vy:
        v.append((i,j))

In [40]:
re_data = []
for i in pos.tolist():
    for t in v:
        d = i.copy()
        d.extend(t)
        re_data.append(d)

In [41]:
len(re_data)

12960

In [42]:
qv = iql.vf(torch.tensor(re_data, dtype=torch.float).cuda())

In [43]:
qv = qv.detach().cpu().numpy()

In [47]:
qv = qv.reshape(-1, len(v)).mean(axis = 1)

In [56]:
qv[:, np.newaxis].shape

(90, 1)

In [58]:
res = np.concatenate((pos_tok, qv[:, np.newaxis]), axis=1)

In [61]:
res = pd.DataFrame(res, columns=['x','y','q'])
res.head()

,x,y,q
0,5.0,8.0,17.925838
1,4.0,7.0,14.955861
2,5.0,7.0,17.363459
3,3.0,7.0,18.724478
4,4.0,8.0,15.740638


In [62]:
res.to_csv('iql_Q_map.csv')